# REFINITIV PEERS AND VALUATION

### Instructions

Please input a valid RIC after running the first and second cell below. Then you can run the rest of the notebook using shift+enter for each cell sequentially. 

This notebook will provide all of the information contained in the popular Eikon/Workspace template of the same name. This includes some general information about the RIC you selected and then various sections on how to calculate Enterprise Value, a financial summary, a financial valuation and finally a drop-down box which allows you to select a 5 year history of over 21 measures.

In [1]:
import refinitiv.data as rd
from refinitiv.data.discovery import Peers
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Box, Layout

%matplotlib inline
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fb4c383fb20 {name='codebook'}>

In [2]:
ric = widgets.Text(
    value='',
    placeholder='Type something',
    description='RIC:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items = [ric]
box = Box(children=items, layout=box_layout)

box

Box(children=(Text(value='', description='RIC:', placeholder='Type something'),), layout=Layout(align_items='s…

### About

In [3]:
df = rd.get_data(ric.value, [
                 "TR.PriceClose.Date", "TR.NetIncome(period=FY0).PED", "TR.NetIncome(period=FI0).PED"])
df.columns = ['Instrument', 'Price Date',
              'Fiscal Year End', 'Fiscal Interim End']
df.transpose()

,0
Instrument,VOD.L
Price Date,2023-07-12 00:00:00
Fiscal Year End,2023-03-31 00:00:00
Fiscal Interim End,2023-03-31 00:00:00


### General Information

In [4]:
df1 = rd.get_data(ric.value, ['TR.BusinessSummary'])
pd.set_option('display.max_colwidth', None)
df1['Business Description'][0]

"Vodafone Group Plc is a United Kingdom-based connectivity and digital services provider. The Company's retail and service operations are split across three broad business lines: Europe Consumer, Vodafone Business and Africa Consumer. The Europe Consumer business provides a range of mobile and fixed-line connectivity services in all of its European markets, enabling customers to call, text and access data on their mobile devices, or access broadband, TV and voice services at home. Vodafone Business serves private and public sector customers of all sizes with a range of connectivity services, supported by its global network. The Africa Consumer business provides a range of mobile services, enabling customers to call, text and access data. It provides digital services, the Internet of Things (IoT) and financial services, that leverage and complement its connectivity business. The Company provides financial services, as well as business, and merchant services in Africa."

### Enterprise Value

In [5]:
df2 = rd.get_data(ric.value, ['TR.NetIncome.PED', 'TR.PriceClose(SDate=0FY)', 'TR.CompanySharesOutstanding', 'TR.CompanyMarketCap', 'TR.CashandSTInvestments',
                              'TR.LTInvestmentAffiliateCompanies', 'TR.TotalDebtOutstanding', 'TR.RedeemablePreferredTtl',
                              'TR.PreferredStockNonRedeemableNet', 'TR.MinorityInterestBSStmt', 'TR.PensionBenefitsUnderfunded'],
                  {'Period': 'FY0'})

df2['TEV'] = df2.iloc[0, 3] + \
    (sum(list(filter(None, df2.iloc[0, 6:10]))) -
     sum(list(filter(None, df2.iloc[0, 4:5]))))

df3 = rd.get_data(ric.value, ['TR.NetIncome.PED', 'TR.PriceClose', 'TR.CompanySharesOutstanding', 'TR.CompanyMarketCap', 'TR.CashandSTInvestments',
                              'TR.LTInvestmentAffiliateCompanies', 'TR.TotalDebtOutstanding', 'TR.RedeemablePreferredTtl',
                              'TR.PreferredStockNonRedeemableNet', 'TR.MinorityInterestBSStmt', 'TR.PensionBenefitsUnderfunded'],
                  {'Period': 'LTM'})

df3['TEV'] = df3.iloc[0, 3] + \
    (sum(list(filter(None, df3.iloc[0, 6:10]))) -
     sum(list(filter(None, df3.iloc[0, 4:5]))))

df2 = pd.concat([df2, df3])
df2.set_index('Period End Date', inplace=True)
df2.transpose()

Period End Date,2023-03-31,2023-07-13
Instrument,VOD.L,VOD.L
Price Close,84.24,72.65
Company Shares,27052982067,27052982067
Company Market Cap,19653991471.675499,19653991471.675499
Cash and Short Term Investments,18722000000,16809000000
LT Investment - Affiliate Companies,11079000000,7730000000
Total Debt,66390000000,70986000000
"Redeemable Preferred Stock, Total",,
"Preferred Stock - Non Redeemable, Net",,
Minority Interest,1084000000,1684000000


### Financial Summary

In [6]:
df4 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FI0).rfperiod', 'TR.TotalRevenue(period=FI0)', 'PERCENT_CHG(TR.TotalRevenue(period=FI0),TR.TotalRevenue(period=FI-1))',
                              'TR.GrossProfit(period=FI0)', 'TR.GrossProfit(period=FI0)/TR.TotalRevenue(period=FI0)',
                              'TR.EBITDA(period=FI0)', 'TR.EBITDAMarginPercent(period=FI0)',
                              'TR.EBIT(period=FI0)', 'TR.EBITMarginPercent(period=FI0)',
                              'TR.NetIncome(period=FI0)', 'TR.NetProfitMargin(period=FI0)',
                              'TR.CapitalExpendituresCFStmt(period=FI0)', 'TR.CapitalExpendituresCFStmt(period=FI0)/TR.TotalRevenue(period=FI0)'])

df4.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df5 = rd.get_data(ric.value, ['TR.TotalRevenue(period=LTM).rfperiod', 'TR.TotalRevenue(period=LTM)', 'PERCENT_CHG(TR.TotalRevenue(period=FI0),TR.TotalRevenue(period=FI-4))',
                              'TR.GrossProfit(period=LTM)', 'TR.GrossMargin(period=LTM)',
                              'TR.EBITDATTM', 'TR.EBITDAMarginPercent(period=LTM)',
                              'TR.EBITTTM', 'TR.EBITMarginPercent(period=LTM)',
                              'TR.NetIncome(period=LTM)', 'TR.NetProfitMargin(period=LTM)',
                              'TR.CapitalExpendituresCFStmt(period=LTM)', 'TR.CapitalExpendituresCFStmt(period=LTM)/TR.TotalRevenue(period=LTM)'])

df5.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df6 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FY-1).rfperiod', 'TR.TotalRevenue(period=FY-1)', 'PERCENT_CHG(TR.TotalRevenue(period=FY-1),TR.TotalRevenue(period=FY-2))',
                              'TR.GrossProfit(period=FY-1)', 'TR.GrossMargin(period=FY-1)',
                              'TR.EBITDA(period=FY-1)', 'TR.EBITDAMarginPercent(period=FY-1)',
                              'TR.EBIT(period=FY-1)', 'TR.EBITMarginPercent(period=FY-1)',
                              'TR.NetIncome(period=FY-1)', 'TR.NetProfitMargin(period=FY-1)',
                              'TR.CapitalExpendituresCFStmt(period=FY-1)', 'TR.CapitalExpendituresCFStmt(period=FY-1)/TR.TotalRevenue(period=FY-1)'])

df6.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']


df7 = rd.get_data(ric.value, ['TR.TotalRevenue(period=FY0).rfperiod', 'TR.TotalRevenue(period=FY0)', 'PERCENT_CHG(TR.TotalRevenue(period=FY0),TR.TotalRevenue(period=FY-1))',
                              'TR.GrossProfit(period=FY0)', 'TR.GrossMargin(period=FY0)',
                              'TR.EBITDA(period=FY0)', 'TR.EBITDAMarginPercent(period=FY0)',
                              'TR.EBIT(period=FY0)', 'TR.EBITMarginPercent(period=FY0)',
                              'TR.NetIncome(period=FY0)', 'TR.NetProfitMargin(period=FY0)',
                              'TR.CapitalExpendituresCFStmt(period=FY0)', 'TR.CapitalExpendituresCFStmt(period=FY0)/TR.TotalRevenue(period=FY0)'])

df7.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df8 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY1).rfperiod', 'TR.RevenueMeanEstimate(period=FY1)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY1),TR.TotalRevenue(period=FY0))",
                              'TR.GrossIncomeMean(period=FY1)', 'TR.GPMMean(period=FY1)',
                              'TR.EBITDAMean(period=FY1)', "TR.EBITDAMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.EBITMean(period=FY1)', "TR.EBITMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.NetIncomeMeanEstimate(period=FY1)', "TR.NetIncomeMeanEstimate(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100",
                              'TR.CAPEXMean(period=FY1)', "TR.CAPEXMean(period=FY1)/TR.RevenueMeanEstimate(period=FY1)*100"])

df8.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df9 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY2).rfperiod', 'TR.RevenueMeanEstimate(period=FY2)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY2),TR.TotalRevenue(period=FY1))",
                              'TR.GrossIncomeMean(period=FY2)', 'TR.GPMMean(period=FY2)',
                              'TR.EBITDAMean(period=FY2)', "TR.EBITDAMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.EBITMean(period=FY2)', "TR.EBITMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.NetIncomeMeanEstimate(period=FY2)', "TR.NetIncomeMeanEstimate(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100",
                              'TR.CAPEXMean(period=FY2)', "TR.CAPEXMean(period=FY2)/TR.RevenueMeanEstimate(period=FY2)*100"])

df9.columns = ['Instrument', 'Fiscal Period', 'Revenue', '  Growth(%)', 'Gross Income', '  Margin(%)',
               'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df10 = rd.get_data(ric.value, ['TR.RevenueMeanEstimate(period=FY3).rfperiod', 'TR.RevenueMeanEstimate(period=FY3)', "PERCENT_CHG(TR.RevenueMeanEstimate(period=FY3),TR.TotalRevenue(period=FY2))",
                               'TR.GrossIncomeMean(period=FY3)', 'TR.GPMMean(period=FY3)',
                               'TR.EBITDAMean(period=FY3)', "TR.EBITDAMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.EBITMean(period=FY3)', "TR.EBITMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.NetIncomeMeanEstimate(period=FY3)', "TR.NetIncomeMeanEstimate(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100",
                               'TR.CAPEXMean(period=FY3)', "TR.CAPEXMean(period=FY3)/TR.RevenueMeanEstimate(period=FY3)*100"])

df10.columns = ['Instrument', 'Fiscal Period', 'Revenue',
                '  Growth(%)', 'Gross Income', '  Margin(%)', 'EBITDA', '  Margin(%)', 'EBIT', '  Margin(%)', 'Net Income', '  Margin(%)', 'Capex', '  % of Sales']

df4 = pd.concat([df4, df5, df6, df7, df8, df9, df10], ignore_index=True)
df4.set_index('Fiscal Period', inplace=True)
df4.transpose()

Fiscal Period,FH0,LTM,FY-1,FY0,FY1,FY2,FY3
Instrument,VOD.L,VOD.L,VOD.L,VOD.L,VOD.L,VOD.L,VOD.L
Revenue,22776000000,45706000000,45580000000,45706000000,43688848530,43707384410,44105265880
Growth(%),-0.671609,1.760343,4.042548,0.276437,-4.413319,NaN,NaN
Gross Income,7506000000,14856000000,15006000000,14856000000,28060666670,28103000000,20551000000
Margin(%),0.329557,32.50491,32.92233,32.50339,32.0,32.0,32.3
EBITDA,9343000000,18941000000,19547000000,18941000000,13390652530,13528632500,13736625000
Margin(%),41.02125,41.439539,42.885037,41.440949,30.650047,30.952739,31.145091
EBIT,2578000000,5323000000,5702000000,5323000000,4832313110,5148318290,5371858930
Margin(%),11.318932,11.645074,12.509873,11.646173,11.060747,11.779058,12.179632
Net Income,10852000000,11838000000,2237000000,11838000000,2449396440,2696645200,2949979360


### Financial Valuation

In [7]:
headers = ['FY0', 'LTM', 'FY1', 'FY2', 'FY3']
index = ['a', 'b']
TEV_to_Sales = pd.Series([df2['TEV'][0]/df4.iloc[3, 1], df2['TEV'][1]/df4.iloc[1, 1], df2['TEV'][1]/df4.iloc[4, 1],
                          df2['TEV'][1]/df4.iloc[5, 1], df2['TEV'][1]/df4.iloc[6, 1]], index=headers)
TEV_to_EBITDA = pd.Series([df2['TEV'][0]/df4.iloc[3, 5], df2['TEV'][1]/df4.iloc[1, 5], df2['TEV'][1]/df4.iloc[4, 5],
                           df2['TEV'][1]/df4.iloc[5, 5], df2['TEV'][1]/df4.iloc[6, 5]], index=headers)
TEV_to_EBIT = pd.Series([df2['TEV'][0]/df4.iloc[3, 7], df2['TEV'][1]/df4.iloc[1, 7], df2['TEV'][1]/df4.iloc[4, 7],
                         df2['TEV'][1]/df4.iloc[5, 7], df2['TEV'][1]/df4.iloc[6, 7]], index=headers)
list_of_series = [TEV_to_Sales, TEV_to_EBITDA, TEV_to_EBIT]
tev = pd.DataFrame(list_of_series, columns=headers)
tev.rename(index={0: 'TEV_to_Sales', 1: 'TEV_to_EBITDA', 2: 'TEV_to_EBIT'})

,FY0,LTM,FY1,FY2,FY3
TEV_to_Sales,1.856824,1.884107,1.971098,1.970262,1.952488
TEV_to_EBITDA,4.480650,4.546486,6.430978,6.365388,6.269006
TEV_to_EBIT,15.943639,16.177905,17.820656,16.726819,16.030762


In [8]:
curn = rd.get_data(ric.value, ['TR.PriceClose.currency'])
curn = curn['Currency'][0].upper()
curn

df11 = rd.get_data(ric.value, ['TR.EBITDA(period=FY0).rfperiod', "TR.EBITDA(period=FY0)/TR.InterestExpenseSupplemental(period=FY0)",
                                    "TR.EBIT(period=FY0)/TR.InterestExpenseSupplemental(period=FY0)",
                                    "(TR.EBITDA(period=FY0)-TR.CapitalExpendituresCFStmt(period=FY0))/(TR.InterestExpenseSupplemental(period=FY0))",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EBITDA(period=FY0)",
                                    "TR.TotalDebtOutstanding(period=FY0)/TR.EV*100",
                                    "TR.PriceClose('"+df2.index[0].strftime('%Y-%m-%d')+"',curn="+curn+")/TR.DilutedEpsExclExtra(Period=FY0)"])

df11.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df12 = rd.get_data(ric.value, ['TR.EBITDA(period=LTM).rfperiod', "TR.EBITDA(period=LTM)/TR.InterestExpenseSupplemental(period=LTM)",
                                    "TR.EBIT(period=LTM)/TR.InterestExpenseSupplemental(period=LTM)",
                                    "(TR.EBITDA(period=LTM)-TR.CapitalExpendituresCFStmt(period=LTM))/(TR.InterestExpenseSupplemental(period=LTM))",
                                    "TR.TotalDebtOutstanding(period=LTM)/TR.EBITDA(period=LTM)",
                                    "TR.TotalDebtOutstanding(period=LTM)/TR.EV*100",
                                    "TR.PriceClose('"+df2.index[0].strftime('%Y-%m-%d')+"',curn="+curn+")/TR.DilutedEpsExclExtra(Period=LTM)"])

df12.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']


df13 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY1).rfperiod', "TR.EBITDAMean(period=FY1)/TR.IntExpMean(period=FY1)",
                                    "TR.EBITMean(period=FY1)/TR.IntExpMean(period=FY1)",
                                    "(TR.EBITMean(period=FY1)-TR.CAPEXMean(period=FY1))/TR.IntExpMean(period=FY1)",
                                    "TR.TotalDebtOutstanding(period=FY1)/TR.EBITDA(period=FY1)",
                                    "TR.TotalDebtOutstanding(period=FY1)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY1)"])

df13.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df14 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY2).rfperiod', "TR.EBITDAMean(period=FY2)/TR.IntExpMean(period=FY2)",
                                    "TR.EBITMean(period=FY2)/TR.IntExpMean(period=FY2)",
                                    "(TR.EBITMean(period=FY2)-TR.CAPEXMean(period=FY2))/TR.IntExpMean(period=FY2)",
                                    "TR.TotalDebtOutstanding(period=FY2)/TR.EBITDA(period=FY2)",
                                    "TR.TotalDebtOutstanding(period=FY2)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY2)"])

df14.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df15 = rd.get_data(ric.value, ['TR.EBITDAMean(period=FY3).rfperiod', "TR.EBITDAMean(period=FY3)/TR.IntExpMean(period=FY3)",
                                    "TR.EBITMean(period=FY3)/TR.IntExpMean(period=FY3)",
                                    "(TR.EBITMean(period=FY3)-TR.CAPEXMean(period=FY3))/TR.IntExpMean(period=FY3)",
                                    "TR.TotalDebtOutstanding(period=FY3)/TR.EBITDA(period=FY3)",
                                    "TR.TotalDebtOutstanding(period=FY3)/TR.EV*100",
                                    "TR.PtoEPSMeanEst(Period=FY3)"])

df15.columns = ['Instrument', 'Fiscal Period', 'EBITDA/Interest Expense', 'EBIT/Interest Expense',
                'EBITDA-Capex/Interest Expense', 'Total Debt/EBITDA', 'Total Debt/EV', 'PE']

df11 = pd.concat([df11, df12, df13, df14, df15], ignore_index=True)
df11.set_index('Fiscal Period', inplace=True)

df11.transpose()

Fiscal Period,FY0,LTM,FY1,FY2,FY3
Instrument,VOD.L,VOD.L,VOD.L,VOD.L,VOD.L
EBITDA/Interest Expense,7.350019,43.442661,8.820448,8.970126,8.859971
EBIT/Interest Expense,2.06558,12.208716,3.183054,3.41358,3.464789
EBITDA-Capex/Interest Expense,3.774932,22.311927,-1.766443,-3.329387,-1.349209
Total Debt/EBITDA,3.505095,3.747743,,,
Total Debt/EV,92.610363,99.021528,NaN,NaN,NaN
PE,2.095211,2.07207,10.032965,9.137541,8.563175


In [9]:
curn = rd.get_data(ric.value, ['TR.PriceClose.currency'])
curn = curn['Currency'][0].upper()
curn

'GBP'

In [10]:
mSel = widgets.Dropdown(
    options=[('Book Value per Share', 'BVPS'), ('Capital Expenditure', 'Capex'), ('Cash Earnings per Share', 'EPSCash'),
             ('Cash Flow per Share', 'CFPS'), ('Dividend per Share',
                                               'DPS'), ('EBIT', 'EBIT'), ('EBITDA', 'EBITDA'),
             ('EBITDA per Share', 'EBITDAPS'), ('Enterprise Value',
                                                'EV'), ('EPS', 'EPS'), ('EPS -Fully Reported', 'EPSFR'),
             ('Funds Flow from Operations', 'FFO'), ('Gross Margin',
                                                     'GPM'), ('Net Asset Value', 'NetAssetValue'),
             ('Net Debt', 'NetDebt'), ('Net Income',
                                       'NetDebt'), ('Operating Profit', 'OProfit'),
             ('Pre-tax Profit', 'PreTaxProfit'), ('Revenue', 'Revenue'), ('ROA', 'ROA'), ('ROE', 'ROE')],
    value='BVPS',
    description='Metric:',
    disabled=False
)

pSel = widgets.Dropdown(
    options=['Annual', 'Interim'],
    value='Annual',
    description='Period:',
    disabled=False
)

cSel = widgets.Dropdown(
    options=['USD', 'EUR', 'GBP', 'CAD', 'RUB', 'CNY', 'INR', 'JPY'],
    value='GBP',
    description='Curn:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items2 = [mSel, pSel, cSel]
box2 = Box(children=items2, layout=box_layout)
box2

Box(children=(Dropdown(description='Metric:', options=(('Book Value per Share', 'BVPS'), ('Capital Expenditure…

In [11]:
if pSel.value == 'Annual':
    df16 = rd.get_data(ric.value, ['TR.'+mSel.value+'ActValue', 'TR.'+mSel.value+'ActReportDate',
                                        'TR.'+mSel.value+'MeanEstimate',
                                        'TR.'+mSel.value+'ActSurprise', 'TR.'+mSel.value+'ActSueScore',
                                        'TR.'+mSel.value+'NumofEstimates'], {'Period': 'FY0', 'Sdate': 'FY0', 'Edate': 'FY-4',
                                                                             'FRQ': 'FY', 'Curn': cSel.value})
else:
    df16 = rd.get_data(ric.value, ['TR.'+mSel.value+'ActValue', 'TR.'+mSel.value+'ActReportDate',
                                        'TR.'+mSel.value+'MeanEstimate',
                                        'TR.'+mSel.value+'ActSurprise', 'TR.'+mSel.value+'ActSueScore',
                                        'TR.'+mSel.value+'NumofEstimates'], {'Period': 'FI0', 'Sdate': 'FI0', 'Edate': 'FI-19',
                                                                             'FRQ': 'FI', 'Curn': cSel.value})
df16

,Instrument,Book Value Per Share - Actual,Report Date,Book Value Per Share - Actual Surprise,Book Value Per Share - Standard Unexpected Earnings,Book Value Per Share - Number of Estimates
0,VOD.L,2.04323,2023-05-16 07:00:00,19.977,6.06435,9
1,VOD.L,1.628454,2022-05-17 05:40:00,-0.116,-0.02489,10
2,VOD.L,1.627955,2021-05-18 07:00:00,-12.052,-1.65379,11
3,VOD.L,1.849106,2020-05-12 07:00:00,-5.032,-0.50147,11
4,VOD.L,2.024913,2019-05-14 07:01:00,-0.343,-0.03725,12


In [17]:
rd.close_session()

**Further Resources for Refinitiv Data Libraries API on Developer Community Portal**

* [Overview](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) 
* [Quick Start ](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start)
* [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
* [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials)
* [Q&A Forums](https://community.developers.refinitiv.com/spaces/321/index.html)